In [1]:
import os
from os import walk

import pandas as pd
import numpy as np
import sys
import re

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
import yaml

from collections import Counter

import sys
sys.path.append('../parsing_and_plotting/')
from parse_data import * 

%matplotlib inline

pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 1500)
pd.set_option('max_colwidth', 400)

In [2]:
#Chose allele resolution: options: "one_field", two_field"
resolution = "two_field"

# Load Typing Results

In [3]:
#Make function for conversion, to be used in this run of the notebook:
p_group_dict = make_p_group_dict()
e_group_dict = make_e_group_dict()

def convert_allele(allele, resolution = 'two_field'):

    """
    input:
    allele (str): An allele in the format (A|B|C|DRB1|DQB1)\*\d{2}:\d{2,3}:?\d{0,3}G?:?\d{0,3} to be converted.
    resolution:   the resolution, with which the allele is converted to

    """
    if resolution == "one_field":
        converted_allele = convert_to_one_field(allele)
    
    elif resolution == "two_field":
        converted_allele = convert_to_two_field(allele)
        
    elif resolution == "p_group":
        converted_allele = convert_to_p_group(allele, p_group_dict=p_group_dict)
    
    elif resolution == "e_group":
        converted_allele = convert_to_e_group(allele, e_group_dict=e_group_dict, p_group_dict=p_group_dict)
            
    else:
        print("A conversion mistake happend. Please specify a correct conversion type.")
        converted_allele = None
        
    return converted_allele      

In [4]:
def define_resultpath(read_length='65', coverage='50', damage='no_dmg'):
    if damage == 'dmg':
        resultpath = f'results/damaged_reads/{read_length}_read_length/{coverage}X_coverage/'
    else:
        resultpath = f'results/no_damage/{read_length}_read_length/{coverage}X_coverage/'

    return resultpath

def load_optitype_results(optitype_result_filepath):

    optitype_files = []
    for (dirpath, dirnames, filenames) in walk(optitype_result_filepath):
        optitype_files.extend(filenames)

    #print(optitype_files)

    optitype_results = dict()

    for filename in optitype_files:

        #Check for right file, and that the file is not empty
        if (filename.endswith('.tsv')) and (os.stat(optitype_result_filepath + filename).st_size != 0):
            temp_results_raw = list(pd.read_csv(optitype_result_filepath + filename, sep = "\t").iloc[0])[1:7]

            temp_results = [i for i in temp_results_raw if isinstance(i,str)]

            #Make dict of dicts for results:
            temp_result_dict = dict()

            for pred in temp_results:
                gene = re.search(r"(A|B|C|DRB1|DQB1)", pred).group(0)

                pred_converted = convert_allele(pred)

                #Add to list of predictions for this sample
                if gene not in temp_result_dict.keys():
                    temp_result_dict[gene] = [[pred_converted]]
                else:
                    temp_result_dict[gene] += [[pred_converted]]

            optitype_results[filename[:-11]] = temp_result_dict

        #Add empty entry for empty file
        elif (filename.endswith('.tsv')) and (os.stat(optitype_result_filepath + filename).st_size == 0):
            optitype_results[filename[:-11]] = dict()

    return optitype_results

In [5]:
# optitype_typing_dict = dict()

# read_lengths = ["13", "15", '20', '25','30','35', '45', '55', '65']
# coverages = ['1', '2','5', '10', '20', "50"]

# for dmg in ['dmg', 'no_dmg']:
#     optitype_typing_dict[dmg] = dict()
#     for rl in read_lengths:
#         optitype_typing_dict[dmg][rl] = dict()
#         for cov in coverages:
#             optitype_typing_dict[dmg][rl][cov] = load_optitype_results(define_resultpath(read_length=rl, coverage=cov, damage=dmg))

# with open('results/optitype_typing_dict.json', 'w') as outfile:
#     json.dump(optitype_typing_dict, outfile)

with open('results/optitype_typing_dict.json', 'r') as infile:
    optitype_typing_dict = json.load(infile)

In [34]:
optitype_typing_dict['no_dmg']['13']['50']

{}

# Load Gold standard data

In [6]:
MG_exome_merged_df = pd.read_pickle("../parsing_and_plotting/result_data/MG_exome_merged_df.pkl")

gs_two_field_df = pd.read_pickle("../parsing_and_plotting/result_data/gs_two_field_df.pkl")

#Only pick the samples included in the aDNA study
with open("config.yaml", "r") as stream:
    config_dict = yaml.safe_load(stream)

gold_standard_id_list = config_dict['sample_id'].keys()

gs_two_field_df = gs_two_field_df.loc[gold_standard_id_list]

# Calculate Results - Typing Accuracy + Call Rate

In [7]:
def validate_call(correct_alleles, predicted_alleles, resolution):
    #Start by converting the alleles to the correct resolution
    correct_call_1 = {convert_allele(allele, resolution=resolution) for allele in correct_alleles[0]}
    correct_call_2 = {convert_allele(allele, resolution=resolution) for allele in correct_alleles[1]}

    pred_1 = {convert_allele(predicted_alleles[0][0], resolution=resolution)}
    pred_2 = {convert_allele(predicted_alleles[1][0], resolution=resolution)}

    try:
        correct_0_pred_0 = list(correct_call_1.intersection(pred_1))
        correct_1_pred_1  = list(correct_call_2.intersection(pred_2))

        option_1 = [correct_0_pred_0, correct_1_pred_1]

        correct_0_pred_1  = list(correct_call_1.intersection(pred_2))
        correct_1_pred_0  = list(correct_call_2.intersection(pred_1))

        option_2 = [correct_0_pred_1, correct_1_pred_0]

        hits_1 = len([i for i in option_1 if i != []])
        hits_2 = len([i for i in option_2 if i != []])

        num_correct_hits = max(hits_1,hits_2)
        
        
    except KeyError as error:
        num_correct_hits = 0
    
    return num_correct_hits

#Get the number of predictions from a tool for a specific allele for a specific subject (2 or 0)
def get_count(tool_prediction):
    try:
        pred = tool_prediction
        return len(pred)
    except KeyError as error:
        return 0

In [28]:
def validate_typing(optitype_typing_dict, gold_standard_df, resolution):
    loci = ['A', 'B', 'C']
    results_dict = dict()
    total_correct_calls = 2 * len(gold_standard_id_list)


    #Loop over parameters
    for dmg, read_length_dict in optitype_typing_dict.items():
        results_dict[dmg] = dict()
        for rl, coverage_dict in read_length_dict.items():
            results_dict[dmg][rl] = dict()
            for cov,_ in coverage_dict.items():
                results_dict[dmg][rl][cov] = dict()
                for locus in loci:
                    results_dict[dmg][rl][cov][locus] = dict()
                    results_dict[dmg][rl][cov][locus]['count'] = 0
                    results_dict[dmg][rl][cov][locus]['score'] = 0
                    
                    #Only include combinations, where all samples had high enough coverage - even when read length was low.
                    for subject in gold_standard_df.index:
                        #If typing exists:
                        if subject in optitype_typing_dict[dmg][rl][cov]:
                            if locus in optitype_typing_dict[dmg][rl][cov][subject]:
                                optitype_pred = optitype_typing_dict[dmg][rl][cov][subject][locus]
                                #Count number of total calls:
                                results_dict[dmg][rl][cov][locus]['count'] += len(optitype_pred)
                            
                                #check whether it is valid.
                                correct_alleles_list = gold_standard_df.loc[subject, locus]
                                predicted_alleles = optitype_typing_dict[dmg][rl][cov][subject][locus]
                                results_dict[dmg][rl][cov][locus]['score'] += validate_call(correct_alleles_list, optitype_pred, resolution)
                        else:

                                
                    results_dict[dmg][rl][cov][locus]['call_rate'] = results_dict[dmg][rl][cov][locus]['count'] * 100  / total_correct_calls
                    results_dict[dmg][rl][cov][locus]['typing_accuracy'] = results_dict[dmg][rl][cov][locus]['score'] * 100 / total_correct_calls

                #Calculate across all class I
                results_dict[dmg][rl][cov]['HLA-I'] = dict()
                results_dict[dmg][rl][cov]['HLA-I']['count'] = sum([results_dict[dmg][rl][cov][locus]['count'] for locus in loci]) 
                results_dict[dmg][rl][cov]['HLA-I']['score'] = sum([results_dict[dmg][rl][cov][locus]['score'] for locus in loci]) 

                results_dict[dmg][rl][cov]['HLA-I']['call_rate'] = results_dict[dmg][rl][cov]['HLA-I']['count'] * 100  / (3 * total_correct_calls)
                results_dict[dmg][rl][cov]['HLA-I']['typing_accuracy'] = results_dict[dmg][rl][cov]['HLA-I']['score'] * 100 / (3 * total_correct_calls)


    return results_dict

In [29]:
resolution = 'two_field'
results_dict = validate_typing(optitype_typing_dict, gold_standard_df=gs_two_field_df, resolution=resolution)

with open(f'results/results_dict_{resolution}.json', 'w') as outfile:
    json.dump(results_dict, outfile)

dmg
13
50
A
HG00096
dmg
13
50
A
HG00259
dmg
13
50
A
HG00262
dmg
13
50
A
HG00268
dmg
13
50
A
HG00419
dmg
13
50
A
HG00641
dmg
13
50
A
HG00731
dmg
13
50
A
HG00732
dmg
13
50
A
HG00734
dmg
13
50
A
HG00737
dmg
13
50
A
HG01048
dmg
13
50
A
HG01049
dmg
13
50
A
HG01051
dmg
13
50
A
HG01054
dmg
13
50
A
HG01055
dmg
13
50
A
HG01066
dmg
13
50
A
HG01094
dmg
13
50
A
HG01110
dmg
13
50
A
HG01111
dmg
13
50
A
HG01112
dmg
13
50
A
HG01168
dmg
13
50
A
HG01756
dmg
13
50
A
HG01757
dmg
13
50
A
HG01872
dmg
13
50
A
HG01873
dmg
13
50
A
HG01886
dmg
13
50
A
HG01953
dmg
13
50
A
HG01968
dmg
13
50
A
HG02014
dmg
13
50
A
HG02057
dmg
13
50
A
NA06986
dmg
13
50
A
NA11918
dmg
13
50
A
NA12275
dmg
13
50
A
NA18522
dmg
13
50
A
NA18853
dmg
13
50
A
NA18870
dmg
13
50
A
NA18912
dmg
13
50
A
NA19102
dmg
13
50
A
NA19130
dmg
13
50
A
NA19172
dmg
13
50
A
NA19238
dmg
13
50
A
NA19334
dmg
13
50
A
NA19346
dmg
13
50
A
NA19347
dmg
13
50
A
NA19471
dmg
13
50
A
NA19474
dmg
13
50
A
NA19625
dmg
13
50
A
NA19648
dmg
13
50
A
NA20313
dmg
13
50
A
NA20502


In [27]:
results_dict

{'dmg': {'13': {'1': {'A': {'count': 100,
     'score': 24,
     'call_rate': 100.0,
     'typing_accuracy': 24.0},
    'B': {'count': 100,
     'score': 17,
     'call_rate': 100.0,
     'typing_accuracy': 17.0},
    'C': {'count': 100,
     'score': 35,
     'call_rate': 100.0,
     'typing_accuracy': 35.0},
    'HLA-I': {'count': 300,
     'score': 76,
     'call_rate': 100.0,
     'typing_accuracy': 25.333333333333332}},
   '2': {'A': {'count': 100,
     'score': 36,
     'call_rate': 100.0,
     'typing_accuracy': 36.0},
    'B': {'count': 100,
     'score': 35,
     'call_rate': 100.0,
     'typing_accuracy': 35.0},
    'C': {'count': 100,
     'score': 53,
     'call_rate': 100.0,
     'typing_accuracy': 53.0},
    'HLA-I': {'count': 300,
     'score': 124,
     'call_rate': 100.0,
     'typing_accuracy': 41.333333333333336}},
   '5': {'A': {'count': 100,
     'score': 57,
     'call_rate': 100.0,
     'typing_accuracy': 57.0},
    'B': {'count': 100,
     'score': 52,
     'cal

# Save all dataframes with results as pickle objects
